In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/necessary-files/config (1).yml
/kaggle/input/necessary-files/step-000029999.ckpt
/kaggle/input/necessary-files/dataparser_transforms (1).json
/kaggle/input/bear-dataset-16-4-25/transforms.json
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00012.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00038.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00023.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00042.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00021.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00037.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00030.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00009.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00008.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00004.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00001.jpg
/kaggle/input/bear-dataset-16-4-25/images/images/frame_00019.jpg
/kaggle/input

In [2]:
pip install nerfstudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of nuscenes-devkit to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.8/580.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 23.3 MB/s eta 0

In [3]:
import os
import shutil

# Create new structure
scene_dir = "/kaggle/working/my_scene"
os.makedirs(scene_dir, exist_ok=True)
os.makedirs(f"{scene_dir}/images", exist_ok=True)

# Copy transforms.json
shutil.copy("/kaggle/input/bear-dataset-16-4-25/transforms.json", scene_dir)

# Copy images from nested images/images/ to just images/
shutil.copytree(
    "/kaggle/input/bear-dataset-16-4-25/images/images",
    f"{scene_dir}/images",
    dirs_exist_ok=True
)


'/kaggle/working/my_scene/images'

In [4]:
import json
from pathlib import Path
import random

# Path to original transforms.json
json_path = Path("/kaggle/working/my_scene/transforms.json")

# Load the original JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Clean file paths
for frame in data["frames"]:
    fname = Path(frame["file_path"]).name
    base = fname.split(".")[0]  # strip after first dot
    frame["file_path"] = f"./images/{base}.jpg"

# Shuffle frames for fair splitting
random.seed(42)
random.shuffle(data["frames"])

# Split sizes
n = len(data["frames"])
n_train = int(0.8 * n)
n_val = int(0.1 * n)
n_test = n - n_train - n_val

# Split
frames_train = data["frames"][:n_train]
frames_val = data["frames"][n_train:n_train+n_val]
frames_test = data["frames"][n_train+n_val:]

# Prepare separate JSONs
base_out = json_path.parent
for name, frames in [("train", frames_train), ("val", frames_val), ("test", frames_test)]:
    out_data = data.copy()
    out_data["frames"] = frames
    out_path = base_out / f"transforms_{name}.json"
    with open(out_path, "w") as f:
        json.dump(out_data, f, indent=4)
    print(f"✅ Saved {name}: {out_path} with {len(frames)} frames")

# Optional: remove original if you want
# json_path.unlink()


✅ Saved train: /kaggle/working/my_scene/transforms_train.json with 36 frames
✅ Saved val: /kaggle/working/my_scene/transforms_val.json with 4 frames
✅ Saved test: /kaggle/working/my_scene/transforms_test.json with 5 frames


In [5]:
import os
import shutil

# Define base paths
working_dir = "/kaggle/working"
scene_dir = os.path.join(working_dir, "my_scene")
output_dir = os.path.join(working_dir, "outputs", "my_scene", "nerfacto", "2025-04-16_115514")
model_dir = os.path.join(output_dir, "nerfstudio_models")

# Create all directories
os.makedirs(scene_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Define source file paths
src_config = "/kaggle/input/necessary-files/config (1).yml"
src_transforms = "/kaggle/input/necessary-files/dataparser_transforms (1).json"
src_ckpt = "/kaggle/input/necessary-files/step-000029999.ckpt"

# Define target paths (renamed)
dst_config = os.path.join(output_dir, "config.yml")
dst_transforms = os.path.join(output_dir, "dataparser_transforms.json")
dst_ckpt = os.path.join(model_dir, "step-000029999.ckpt")

# Copy and rename the files
shutil.copyfile(src_config, dst_config)
shutil.copyfile(src_transforms, dst_transforms)
shutil.copyfile(src_ckpt, dst_ckpt)

print("✅ Directory structure and files successfully reconstructed.")


✅ Directory structure and files successfully reconstructed.


In [6]:
# !ns-train nerfacto --data /kaggle/working/my_scene --viewer.quit-on-train-completion True


In [7]:
!ns-render camera-path \
  --load-config /kaggle/working/outputs/my_scene/nerfacto/2025-04-16_115514/config.yml \
  --camera-path-filename /kaggle/input/cam-path/spiral_camera_path.json


2025-04-17 18:00:24.654718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744912825.133811     130 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744912825.229167     130 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torc

In [8]:
# !ns-render camera-path \
#   --load-config /kaggle/working/outputs/my_scene/nerfacto/2025-04-16_115514/config.yml \
#   --camera-path-filename /kaggle/working/my_scene/transforms.json \
#   --output-path /content/renders \
#   --output-format images \
#   --image-format png \
#   --rendered-output-names rgb


In [9]:
# rm -rf /kaggle/working/my_scene/transforms.json

In [10]:
import json
import numpy as np

# Load the original transforms
with open("/kaggle/input/bear-dataset-16-4-25/transforms.json") as f:
    transforms = json.load(f)

# Estimate scene center
camera_positions = [np.array(f["transform_matrix"])[:3, 3] for f in transforms["frames"]]
camera_positions = np.array(camera_positions)
look_at = np.mean(camera_positions, axis=0)

# Parameters
num_frames = 90
radius = 1.5
fixed_z = look_at[2] + 0.5  # Adjust height above/below bear
seconds = 6
fov = 60
render_width = 800
render_height = 800
up = np.array([0.0, 0.0, 1.0])  # Up is Z-axis now!

def normalize(v):
    return v / np.linalg.norm(v)

def create_camera_to_world(position, look_at, up):
    forward = normalize(look_at - position)
    right = normalize(np.cross(forward, up))
    up_corrected = np.cross(right, forward)
    c2w = np.eye(4)
    c2w[:3, 0] = right
    c2w[:3, 1] = up_corrected
    c2w[:3, 2] = -forward
    c2w[:3, 3] = position
    return c2w

# Generate circular path in XY
frames = []
for i in range(num_frames):
    angle = 2 * np.pi * i / num_frames
    x = look_at[0] + radius * np.cos(angle)
    y = look_at[1] + radius * np.sin(angle)
    z = fixed_z

    position = np.array([x, y, z])
    c2w = create_camera_to_world(position, look_at, up)

    frames.append({
        "camera_to_world": c2w.tolist(),
        "fov": fov
    })

spiral_camera_path = {
    "version": "1.0",
    "seconds": seconds,
    "render_height": render_height,
    "render_width": render_width,
    "camera_path": frames
}

# Save the updated path
spiral_path_file = "/kaggle/working/spiral_camera_path.json"
with open(spiral_path_file, "w") as f:
    json.dump(spiral_camera_path, f, indent=2)

spiral_path_file


'/kaggle/working/spiral_camera_path.json'

In [11]:
!ns-export camera-path --load-config /kaggle/working/outputs/my_scene/nerfacto/2025-04-16_091427/config.yml --output-path /kaggle/working/spiral_path.json --trajectory spiral


2025-04-17 18:01:23.011723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744912883.039229     156 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744912883.047223     156 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torc